In [4]:
import torch as T
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

class DeepQNetwork(nn.Module):
    #Building the constructor for the class
    def __init__(self, ALPHA):
        super(DeepQNetwork, self).__init__()
        self.conv1= nn.Conv2d(1,32,8, stride=4, padding=1)
        self.conv2= nn.Conv2d(32, 64, 4, stride=2)
        self.conv3= nn.Conv2d(64, 128, 4)
        self.fc1 = nn.Linear(128*19*8, 512)
        self.fc2 = nn.Linear(512,6)
        
        self.optimizer = optim.RMSprop(self.parameters(), lr = ALPHA)
        self.loss = nn.MSEloss()
        self.device = T.device('cuda:0' if T.cuda.is_avaliable() else'cpu')
        self.to(self.device)
        
    #feeding the information of the frames to the CNN layers and prop forward 
    def forward(self, observation):
        observation =T.Tensor(observation).to(self.device)
        observation = observatio.view(-1,1,185, 95)
        observation = F.relu(self.conv1(observation))
        observation = F.relu(self.conv2(observation))
        observation = F.relu(self.conv3(observation))
        observation = observation.view(-1, 128*19*8)
        observation = F.relu(self.fc1(observation))
                               
        observation = self.fc2(observation)
        #return the predicted set of actions to take 
        return actions
    
    
#defines the class for the agent 
class Agent(object):
    def __intit__(self, gamma, epsilon, alpha,
                 maxMemorySize, epsEnd=0.05,
                 replace=10000, actionSpace=[0,1,2,3,4,5]):
        slef.GAMMA =gamma
        self.EPSILON =epsilon 
        sel.EPS_END=epsEND
        self.actionSpace = actionSpace
        self.memSize = maxMemorySize
        self.steps = 0
        self.learn_step_counter = 0
        self.memory = []
        self.memCntr=0
        self.replace_target_cnt = replace
        #set the Conv net to the self.Q_eval and self.Q_next
        self.Q_eval = DeepQNetwork(alpha)
        self.Q_next = DeepQNetwork(alpha)
    
    #Encodes the memory portion of the agent.
    def storeTransition(self, state, action, reward, state_):
        if self.memCntr < self.memSize:
            self.memory.append([state, action, reward, state_])
        else:
            self.memory[self.memCntr%self.memSize]=[state, action, reward, state_]
        self.memCntr +=1
        
    
    #Takes the arg max based on the eplsion greedy function.
    #This controls how the agent is greedy and explores 
    def chooseAction(self, observation):
        rand = np.random.random()
        actions = self.q_eval.forward(observation)
        if rand < 1-self.EPSILON:
            action = T.argamax(actions[1]).item()
        else:
            action =np.random.choice(self.actionSpace)
        self.steps +=1
        return action
    
    #Code block on how the agent actually learns from the process. Rand sample with batch 
    def learn(self, batch_size):
        self.Q_eval.optimize.zero_grad()
        if self.replace_target_cnt is not None and self.learn_step_counter% self.repace_target_cnt ==0:
            self.Qnext.load_state_dict(self.Q_eval.state_dict())
        
        if self.memCntr + batch_size <self.mem:
            memStart = int(np.random.choice(range(self.memCntr)))
        else:
            memStart = int(np.random.choice(range(self.memCntr+batch_size-1)))
        miniBatch = self.memory[memStart:memStart+batch_size]
        memory = np.array(miniBatch)
        
        Qpred = self.Q_eval.forward(list(memory[:,0][:])).to(self.Q_eval.device)
        Qnext = self.Q_next.forward(list(memory[:, 3][:])).to(self.Q_eval.device)
        
        maxA = T.argmax(Qnext, dim=1).to(self.Q_eval.device)
        rewards = T.Tensor(list(memory[:,2])).to(self.Q_eval.device)
        Qtarget = Qpred
        Qtarget[:, maxA]= rewards+self.GAMMA*T.max(Qnext[1])
        
        if self.steps>500:
            if self.EPSILON -1e4 >self.EPS_END:
                self.EPSILON -=1e4
            else:
                self.EPSILON =self.EPS_END
            
        loss= self.Q_eval.loss(Qtarget,Qpred).to(self.Q_eval.device)
        loss.backward()
        
        self.Q_eval.optimizer.step()
        self.learn_step_counter+=1      
        
        
        